<a href="https://colab.research.google.com/github/zhangsushen1992/chemical_vae/blob/master/chemvae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-07-11 21:09:54--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-07-11 21:09:54--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
^C
Solving environment: ...working... done

# All requested packages already installed.


real	0m21.461s
user	0m7.882s
sys	

In [ ]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import sys
sys.path.append(".")
import os
print(os.getcwd())
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
import argparse
import numpy as np
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Lambda
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.core import Dense, Flatten, RepeatVector, Dropout
from keras.layers.convolutional import Convolution1D
from keras.layers.recurrent import GRU
from keras.layers.normalization import BatchNormalization
# from chemvae import hyperparameters_1
# from chemvae.train_vae import vectorize_data
from keras.models import Sequential
from keras.layers import InputLayer
import copy
import json
from keras.callbacks import CSVLogger, LambdaCallback
import yaml
import rdkit
import mol_utils as mu


/content


Using TensorFlow backend.


In [ ]:
def vectorize_data(params):
    # @out : Y_train /Y_test : each is list of datasets.
    #        i.e. if reg_tasks only : Y_train_reg = Y_train[0]
    #             if logit_tasks only : Y_train_logit = Y_train[0]
    #             if both reg and logit_tasks : Y_train_reg = Y_train[0], Y_train_reg = 1
    #             if no prop tasks : Y_train = []

    MAX_LEN = params['MAX_LEN']

    CHARS = yaml.safe_load(open(params['char_file']))
    params['NCHARS'] = len(CHARS)
    NCHARS = len(CHARS)
    CHAR_INDICES = dict((c, i) for i, c in enumerate(CHARS))
    #INDICES_CHAR = dict((i, c) for i, c in enumerate(CHARS))

    ## Load data for properties
    if params['do_prop_pred'] and ('data_file' in params):
        if "data_normalization_out" in params:
            normalize_out = params["data_normalization_out"]
        else:
            normalize_out = None

        ################
        if ("reg_prop_tasks" in params) and ("logit_prop_tasks" in params):
            smiles, Y_reg, Y_logit = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    reg_tasks=params['reg_prop_tasks'], logit_tasks=params['logit_prop_tasks'],
                    normalize_out = normalize_out)
        elif "logit_prop_tasks" in params:
            smiles, Y_logit = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    logit_tasks=params['logit_prop_tasks'], normalize_out=normalize_out)
        elif "reg_prop_tasks" in params:
            smiles, Y_reg = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    reg_tasks=params['reg_prop_tasks'], normalize_out=normalize_out)
        else:
            raise ValueError("please sepcify logit and/or reg tasks")

    ## Load data if no properties
    else:
        smiles = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN)

    if 'limit_data' in params.keys():
        sample_idx = np.random.choice(np.arange(len(smiles)), params['limit_data'], replace=False)
        smiles=list(np.array(smiles)[sample_idx])
        if params['do_prop_pred'] and ('data_file' in params):
            if "reg_prop_tasks" in params:
                Y_reg =  Y_reg[sample_idx]
            if "logit_prop_tasks" in params:
                Y_logit =  Y_logit[sample_idx]

    print('Training set size is', len(smiles))
    print('first smiles: \"', smiles[0], '\"')
    print('total chars:', NCHARS)

    print('Vectorization...')
    X = mu.smiles_to_hot(smiles, MAX_LEN, params[
                             'PADDING'], CHAR_INDICES, NCHARS)

    print('Total Data size', X.shape[0])
    if np.shape(X)[0] % params['batch_size'] != 0:
        X = X[:np.shape(X)[0] // params['batch_size']
              * params['batch_size']]
        if params['do_prop_pred']:
            if "reg_prop_tasks" in params:
                Y_reg = Y_reg[:np.shape(Y_reg)[0] // params['batch_size'] * params['batch_size']]
            if "logit_prop_tasks" in params:
                Y_logit = Y_logit[:np.shape(Y_logit)[0] // params['batch_size'] * params['batch_size']]

    np.random.seed(params['RAND_SEED'])
    rand_idx = np.arange(np.shape(X)[0])
    np.random.shuffle(rand_idx)

    TRAIN_FRAC = 1 - params['val_split']
    num_train = int(X.shape[0] * TRAIN_FRAC)

    if num_train % params['batch_size'] != 0:
        num_train = num_train // params['batch_size'] * \
            params['batch_size']

    train_idx, test_idx = rand_idx[: int(num_train)], rand_idx[int(num_train):]

    if 'test_idx_file' in params.keys():
        np.save(params['test_idx_file'], test_idx)

    X_train, X_test = X[train_idx], X[test_idx]
    print('shape of input vector : {}', np.shape(X_train))
    print('Training set size is {}, after filtering to max length of {}'.format(
        np.shape(X_train), MAX_LEN))

    if params['do_prop_pred']:
        # !# add Y_train and Y_test here
        Y_train = []
        Y_test = []
        if "reg_prop_tasks" in params:
            Y_reg_train, Y_reg_test = Y_reg[train_idx], Y_reg[test_idx]
            Y_train.append(Y_reg_train)
            Y_test.append(Y_reg_test)
        if "logit_prop_tasks" in params:
            Y_logit_train, Y_logit_test = Y_logit[train_idx], Y_logit[test_idx]
            Y_train.append(Y_logit_train)
            Y_test.append(Y_logit_test)

        return X_train, X_test, Y_train, Y_test

    else:
        return X_train, X_test


In [ ]:
import json
from collections import OrderedDict


def load_params(param_file=None, verbose=True):
    # Parameters from params.json and exp.json loaded here to override parameters set below
    if param_file is not None:
        hyper_p = json.loads(open(param_file).read(),
                             object_pairs_hook=OrderedDict)
        if verbose:
            print('Using hyper-parameters:')
            for key, value in hyper_p.items():
                print('{:25s} - {:12}'.format(key, str(value)))
            print('rest of parameters are set as default')
    parameters = {

        # for starting model from a checkpoint
        'reload_model': False,
        'prev_epochs': 0,

        # general parameters
        'batch_size': 1,
        'epochs': 1,
        'val_split': 0.1, #validation split
        'loss': 'categorical_crossentropy', # set reconstruction loss

        # convolution parameters
        'batchnorm_conv': True,
        'conv_activation': 'tanh',
        'conv_depth': 2, #4,
        'conv_dim_depth': 8,
        'conv_dim_width': 8,
        'conv_d_growth_factor': 1,#1.15875438383,
        'conv_w_growth_factor': 1,#1.1758149644,

        # decoder parameters
        'gru_depth': 2,#4,
        'rnn_activation': 'tanh',
        'recurrent_dim': 35,
        'do_tgru': False,                # use custom terminal gru layer 
        'terminal_GRU_implementation': 0, # use CPU intensive implementation; other implementation modes (1 - GPU, 2- memory) are not yet implemented
        'tgru_dropout': 0.0,
        'temperature': 1.00,            # amount of noise for sampling the final output 

        # middle layer parameters 
        'hg_growth_factor': 1,#1.4928245388, # growth factor applied to determine size of next middle layer.
        'hidden_dim': 50,#100,
        'middle_layer': 2,
        'dropout_rate_mid': 0.0,
        'batchnorm_mid': True,          # apply batch normalization to middle layers
        'activation': 'tanh',

        # Optimization parameters
        'lr': 0.000312087049936,
        'momentum': 0.936948773087,
        'optim': 'adam',                # optimizer to be used

        # vae parameters
        'vae_annealer_start': 22,       # Center for variational weigh annealer 
        'batchnorm_vae': False,         # apply batch normalization to output of the variational layer
        'vae_activation': 'tanh',
        'xent_loss_weight': 1.0,        # loss weight to assign to reconstruction error.
        'kl_loss_weight': 1.0,          # loss weight to assing to KL loss
        "anneal_sigmod_slope": 1.0,     # slope of sigmoid variational weight annealer
        "freeze_logvar_layer": False,   # Choice of freezing the variational layer until close to the anneal starting epoch
        "freeze_offset": 1,             # the number of epochs before vae_annealer_start where the variational layer should be unfrozen

        # property prediction parameters:
        'do_prop_pred': False,          # whether to do property prediction
        'prop_pred_depth': 3,
        'prop_hidden_dim': 36,
        'prop_growth_factor': 0.8,      # ratio between consecutive layer in property prediction
        'prop_pred_activation': 'tanh', 
        'reg_prop_pred_loss': 'mse',    # loss function to use with property prediction error for regression tasks
        'logit_prop_pred_loss': 'binary_crossentropy',  # loss function to use with property prediction for logistic tasks 
        'prop_pred_loss_weight': 0.5,
        'prop_pred_dropout': 0.0,
        'prop_batchnorm': True,

        # print output parameters
        "verbose_print": 0,

    }
    # overwrite parameters
    parameters.update(hyper_p)
    return parameters


In [ ]:
def layer_model(params):
    '''
    This function sets up the model of autoencoders. It consists of encoder layers (Conv1D) defined by 'conv_depth', 
    and decoder layers defined by 'gru_depth'. There are middle layers defined by 'middle_layer' which are fully 
    connected layers defined by 1*1 Conv1D layers.
    '''
    totallayer=(params['conv_depth']*2+3+params['middle_layer']*3*2+params['gru_depth'])
    layer=[None]*totallayer
    # ----------------- Input layer-----------------
    layer[0] = keras.layers.InputLayer(input_shape=(params['MAX_LEN'], params['NCHARS']), name='input_molecule_smi')
    # -----------------Encoder layers with batch normalisation-----------------
    layer[1] = Convolution1D(int(params['conv_dim_depth'] *
                          params['conv_d_growth_factor']),
                      int(params['conv_dim_width'] *
                          params['conv_w_growth_factor']),
                      activation='tanh',
                      name="encoder_conv0")
    if params['batchnorm_conv']:
        layer[2] = BatchNormalization(axis=-1, name="encoder_norm0")

    for j in range(1, params['conv_depth']):
        layer[1+j*2] = Convolution1D(int(params['conv_dim_depth'] *
                            params['conv_d_growth_factor'] ** (j)),
                            int(params['conv_dim_width'] *
                            params['conv_w_growth_factor'] ** (j)),
                            activation='tanh',
                            name="encoder_conv{}".format(j))
        if params['batchnorm_conv']:
            layer[2+j*2] = BatchNormalization(axis=-1,
                                   name="encoder_norm{}".format(j))
    n=params['conv_depth']*2+1

    # -----------------Flatten the middle layers to input to GRUS-----------------
    layer[n]=Flatten(name='encoder_flatten')
    #-----------------Repeat the vectors to match the length required for GRU-----------------
    layer[n+1] = RepeatVector(params['MAX_LEN'],name='decoder_repeat')
    n=n+2

    #-----------------Decoder defined through GRU layers -----------------
    if params['gru_depth'] > 1:
        layer[n] = GRU(params['recurrent_dim'],
                    return_sequences=True, activation='tanh',
                    name="decoder_gru0")

        for k in range(params['gru_depth'] - 2):
            layer[n+1+k] = GRU(params['recurrent_dim'],
                        return_sequences=True, activation='tanh',
                        name="decoder_gru{}".format(k + 1))

        if params['do_tgru']:
            layer[n+params['gru_depth']-1] = TerminalGRU(params['NCHARS'],
                                rnd_seed=params['RAND_SEED'],
                                recurrent_dropout=params['tgru_dropout'],
                                return_sequences=True,
                                activation='softmax',
                                temperature=0.01,
                                name='decoder_tgru',
                                implementation=params['terminal_GRU_implementation'])([x_dec, true_seq_in])
        else:
            layer[n+params['gru_depth']-1]= GRU(params['NCHARS'],
                        return_sequences=True, activation='softmax',
                        name='decoder_gru_final')

    else:
        
        layer[n+1] = GRU(params['NCHARS'],
                    return_sequences=True, activation='softmax',
                    name='decoder_gru_final'
                    )
    #-----------------Return the model as a list of different layers objects-----------------
    return layer
   

In [ ]:
def Cascade(params, training_layer, X_train, Y_train, X_test, Y_test,epochs=20,loss='categorical_crossentropy',
                        #optimizer='sgd',initialLr=0.01,
                        weightDecay=10e-4,patience=10,
                        windowSize=5,batch_size=128,outNeurons=64,nb_classes=10,index=0,fast=True,gradient=False):
    '''
    This function performs cascade learning to the autoencoders defined in the argument training_layer
    '''
    #-----------------Define key variables-----------------
    nextModelToTrain = list() 
    saveEncoderLayersIndexes = list() 
    saveDecoderLayersIndexes = list()
    saveFCEncoderLayersIndexes = list() 
    saveFCDecoderLayersIndexes = list()
    history = dict()
 
    #-----------------Append layer number of key layers (convolutioinal layers) to become a list -----------------
    # This is necessary because we treat batchnormalisation and dropout layers as an integral to convolutional layers
    # Each time a convolutional layer is added, the batchnormalisation and dropout layers are added together.
    # Therefore, saveEncoderLayerIndexes for example, keeps track of the layer number of all convoutional layers such that
    # trianing is performed with the addition of a whole block of layers.
    i = 0
    print(training_layer)
    exit()
    for currentLayer in training_layer: 
        if (currentLayer.name[0:12] == 'encoder_conv'):
            saveEncoderLayersIndexes.append(i)
        if (currentLayer.name[0:15] == 'encoder_flatten'):
            saveFCEncoderLayersIndexes.append(i)
        if (currentLayer.name[0:14] == 'decoder_repeat'):
            saveFCDecoderLayersIndexes.append(i)
        if (currentLayer.name[0:11] == 'decoder_gru'):
            saveDecoderLayersIndexes.append(i)
        i += 1


    #-----------------The model starts to append blocks of layers-----------------
    for i in range(0,len(saveEncoderLayersIndexes)): 
        if ('iter' + str(i) not in history.keys()): 
            history['iter' + str(i)] = dict() 

        if(i == 0):
            for j in training_layer[0:saveEncoderLayersIndexes[1]]: #FOR CORRESPONDING LAYERS FOR CURRENT RUN IN MODEL
                nextModelToTrain.append(j)
            for j in training_layer[saveFCEncoderLayersIndexes[0]:saveFCDecoderLayersIndexes[0]]: #FOR CORRESPONDING LAYERS FOR CURRENT RUN IN MODEL
                nextModelToTrain.append(j)
       
            for j in training_layer[saveFCDecoderLayersIndexes[0]:saveDecoderLayersIndexes[0]]: #FOR CORRESPONDING LAYERS FOR CURRENT RUN IN MODEL
                nextModelToTrain.append(j)
            for j in training_layer[saveDecoderLayersIndexes[-1]:saveDecoderLayersIndexes[-2]:-1]: #FOR CORRESPONDING LAYERS FOR CURRENT RUN IN MODEL
                nextModelToTrain.append(j)

        else:

            for k in training_layer[saveEncoderLayersIndexes[i]+1:saveEncoderLayersIndexes[i]-1:-1]: #GET THE LAYERS OF NEXT MODEL TO TRAIN
                    nextModelToTrain.insert(1+i*2,k)
            j = training_layer[saveDecoderLayersIndexes[-i-1]] #GET THE LAYERS OF NEXT MODEL TO TRAIN
            nextModelToTrain.insert(-i,j)

        nextModelToTrainInputs = Input(shape=(params['MAX_LEN'], params['NCHARS']), name='input_smi')
        
        #-----------------Starts to add layers to the sequential model-----------------

        ModelToTrain=Sequential()
   
        for index,currentlayer in enumerate(nextModelToTrain[:]):
            print('Currently adding',currentlayer)
            ModelToTrain.add(currentlayer)
        
        #-----------------Setting the newly added layers to be trainable -----------------
        #-----------------and the already present layers to be not trainable-----------------
        if i==0:
            for layer in ModelToTrain.layers:
                layer.trainable= True
        else:
            for layer in ModelToTrain.layers[i:i*2]:
                layer.trainable=False
            for layer in ModelToTrain.layers[i*2+2:-i-1]:
                layer.trainable=False
            for layer in ModelToTrain.layers[-i:]:
                layer.trainable=False

        #-----------------Compiling the model-----------------
        optimizer = keras.optimizers.Adam(lr=0.01)
        ModelToTrain.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])

        model_train_targets = {'x_pred':X_train,
                              'z_mean_log_var':np.ones((np.shape(X_train)[0], params['hidden_dim'] * 2))}
        model_test_targets = {'x_pred':X_test,
                              'z_mean_log_var':np.ones((np.shape(X_test)[0], params['hidden_dim'] * 2))}
        
        # ----------------- Fitting of the model begins here-----------------
        print('Fitting begins .....')
        csv_logger = CSVLogger('training.csv',append=True)
        tmpHistory=ModelToTrain.fit(X_train, X_train,
                            batch_size=params['batch_size'],
                            epochs=params['epochs'],
                            initial_epoch=params['prev_epochs'],
                            callbacks=[csv_logger],
                            verbose=params['verbose_print'],
                            validation_data=[X_test, X_test])
        
        #-----------------Printing key results of the model-----------------
        print('Weigths are', ModelToTrain.get_weights())

        print(ModelToTrain.summary())
        history['iter'+str(i)]['lossTraining'] = tmpHistory.history['loss']
        history['iter'+str(i)]['accuracyTraining'] = tmpHistory.history['acc']
       
        print('iter',str(i), history['iter'+str(i)])
        
    return ModelToTrain, history


In [ ]:
def main_run(params):
    X_train, X_test, Y_train, Y_test = vectorize_data(params)
    auto_layer = layer_model(params)
    print('Model is',auto_layer)
    model, history = Cascade(params, auto_layer, X_train, Y_train, X_test, Y_test)

In [ ]:
params = load_params('/content/exp_1.json')
main_run(params)

NameError: ignored